In [2]:
!pip install pandas

     |████████████████████████████████| 10.1MB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 512kB 51.9MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 50.4MB/s eta 0:00:01


In [4]:
import pandas as pd
import numpy as np
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge folium --yes
import folium

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /srv/conda/envs/notebook

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.0.1               |           py37_0         727 KB  conda-forge
    asn1crypto-0.24.0          |        py37_1003         154 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    cffi-1.12.3                |   py37h8022711_0         218 KB  conda-forge
    chardet-3.0.4              |        py37_1003         167 KB  conda-forge
    cryptography-2.6.1         |   py37h72c5cf5_0         607 KB  conda-forge
    folium-0.8.3               |             py_0          54 KB  conda-forge
    idna-2.8                   |        py37_1000         100 KB  conda-forge
    libblas-3.8.0              |       8_openblas           6 KB  conda-forge
    libcblas-3.8.

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/ninjananjo/Clothing-Store-Competition/master/StoreLocations.csv")
df.shape

(8, 56)

In [7]:
df = df[['TOWN','NAME','ID','COUNTRY','Address1','Postcode','Host']]
#df = df[df['ID'].str.startswith("TS")]
df.head(10)

,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host
0,LONDON (OXFORD CIRCUS),LONDON OXFORD CIRCUS,MS0325,ENG,"Ground Floor, 36-38 Great Castle Street",W1W 8LG,TS0001
1,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TM7329,ENG,214 Oxford Street,W1W 8LG,TS0001
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873
4,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE OUTFIT,OU0207,ENG,72-76,NE1 7DF,OU0207
5,Z USA LOS ANGELES,LOS ANGELES,TM8104,US,189 The Grove Drive,90036,TS1271
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012
7,HAYES,HAYES LOMBARDY RET PARK OUTFIT,OU0201,ENG,Unit 3,UB3 3EX,OU0201


In [8]:
#Add lontitude and lattitude data
addresses = df.loc[:,'Postcode']
addresses_alt = df.loc[:,'NAME']
latitude = []
longitude = []

for address, address_alt in zip(addresses, addresses_alt):
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address+", UK")
    location_alt = geolocator.geocode(address_alt+", UK")
    try:
        #print('a')
        latitude.append(location.latitude)
        longitude.append(location.longitude)        
    
    except:
        
        try:
            #print('b')
            latitude.append(location_alt.latitude)
            longitude.append(location_alt.longitude)
            
        except:    
            #print('c')
            latitude.append(0)
            longitude.append(0)
            
        
df.loc[:,'Latitude']= latitude
df.loc[:,'Longitude']= longitude

In [38]:
lati, lngi = getLocation('LONDON OXFORD CIRCUS','W1W 8LG','ENG')
print(lati)
print(lngi)

51.5155336
-0.141352


In [37]:
def getLocation(storeName, postcode, country):
    
    if country == 'ENG': country = "UK"
    
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(postcode+" "+country)
    location_alt = geolocator.geocode(storeName+" "+country)
    try:
        lat = location.latitude
        lng = location.longitude
    except:
        try:
            lat = location_alt.latitude
            lng = location_alt.longitude
        except:
            lat = 0
            lng = 0
            
    return(lat,lng)

In [9]:
#remove regions that location information was not available
df = df[(df['Latitude']!=0)| (df['Longitude']!=0)]
print(df.shape[0])
df

7


,TOWN,NAME,ID,COUNTRY,Address1,Postcode,Host,Latitude,Longitude
0,LONDON (OXFORD CIRCUS),LONDON OXFORD CIRCUS,MS0325,ENG,"Ground Floor, 36-38 Great Castle Street",W1W 8LG,TS0001,51.515534,-0.141352
1,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TM7329,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
2,LONDON (OXFORD CIRCUS),OXFORD CIRCUS,TS0001,ENG,214 Oxford Street,W1W 8LG,TS0001,51.515534,-0.141352
3,LONDON (CITY),CHEAPSIDE,TS0873,ENG,Unit Msu1,EC2V 6AH,TS0873,51.514098,-0.094914
4,NEWCASTLE UPON TYNE,NEWCASTLE UPON TYNE OUTFIT,OU0207,ENG,72-76,NE1 7DF,OU0207,54.975903,-1.612382
6,MANCHESTER (TRAFFORD C),TRAFFORD CENTRE,TS1012,ENG,130-134 Regent Crescent,M17 8AF,TS1012,53.462559,-2.346119
7,HAYES,HAYES LOMBARDY RET PARK OUTFIT,OU0201,ENG,Unit 3,UB3 3EX,OU0201,51.516683,-0.405498


In [7]:
CLIENT_ID = 'QKP4KVQQZPVXMOKNIXHGQAT1E2IJZCHQPFB4XJGP3MC0CFUO' # your Foursquare ID
CLIENT_SECRET = '1N5BIRBR1EA5CO1QFMNHU1241SOFM5KJLLQRJGMDC0NOYWPH' # your Foursquare Secret
VERSION = '20190501' # Foursquare API version
LIMIT = 400 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
categoryId = '4bf58dd8d48988d1f2941735'

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    
    region_list = []
    
    for name, lat, lng in zip(names, latitudes, longitudes):
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(    
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryId)
        
        #print(url) 
        
        # make the GET request
        results = requests.get(url).json()["response"]
        
        #flatten JSON file
        resultsVen = json_normalize(results['venues'])
        resultsCat = json_normalize(data=results['venues'], record_path=['categories'], record_prefix='cat_', errors='ignore')
        resultsVenCat = pd.concat([resultsVen, resultsCat], axis=1)
        resultsVenCat = resultsVenCat[['id','name','location.distance','location.formattedAddress','location.lat','location.lng','cat_name']]
        
        #Add region name and average price 
        for i in range(0,resultsVen.shape[0]):
            region_list.append(name)
                    
        region_list = pd.DataFrame(region_list, columns=['Store ID'])
        
        nearbyVenues = pd.concat([region_list, resultsVenCat], axis=1)

    return(nearbyVenues)

In [9]:
regions_venues = pd.DataFrame()

#request venue information for each region and store in regions_venues
for region in df['ID'].unique():
    region_venues = df[df['ID']==region]
    regions_venues = regions_venues.append((getNearbyVenues(names=region_venues['ID'],
                                                  latitudes=region_venues['Latitude'],
                                                  longitudes=region_venues['Longitude'],
                                                  radius=radius)),ignore_index=True)
    
#rename columns
regions_venues.rename(columns={'id':'Venue ID',
                          'name':'Venue Name',
                          'location.distance':'Venue Distance',
                          'location.formattedAddress':'Venue Address',
                          'location.lat':'Venue Lat',
                          'location.lng':'Venue Lng',
                          'cat_name':'Venue Category'}, inplace=True)

print(regions_venues.shape[0])

113


In [12]:
jdsports = regions_venues[regions_venues['Venue Name']=='JD Sports']
jdsports.to_csv('jdsports_proximty.csv')
jdsports

,Store ID,Venue ID,Venue Name,Venue Distance,Venue Address,Venue Lat,Venue Lng,Venue Category
2,TS0001,4ac518f1f964a520ecad20e3,JD Sports,127,"[197-203 Oxford Street, West End, Greater Lond...",51.515239,-0.139567,Sporting Goods Shop
13,TS0001,5895ef8201f0776e38be9d23,JD Sports,579,"[385-389 Oxford St, Mayfair, Greater London, U...",51.514111,-0.149394,Sporting Goods Shop
58,TS0873,4ccc1f952dc4370440aac408,JD Sports,2331,"[55 Chapel Street Market, Islington, Greater L...",51.533512,-0.107551,Sporting Goods Shop
95,TS1012,4edd380cbe7be2833b142d45,JD Sports,482,"[48 Regents Crescent, Unit L12, Trafford Centr...",53.466613,-2.348693,Sporting Goods Shop


In [ ]:
#Which TM/TS stores do not have a JD Sports withing a 2km radius?
#Add map feature with TM/TS as centre point and surrounding JD Sports.